In [1]:
!pip install datasets
!pip install transformers==4.37.0
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import torch
import pandas as pd
import numpy as np
import gc
import nltk
nltk.download('punkt')
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from nltk.translate.bleu_score import sentence_bleu

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda:0,2" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-0.5B-Chat",torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", padding_side = "left")

# mainmodel = AutoModelForCausalLM.from_pretrained(
#     "Qwen/Qwen1.5-0.5B-Chat",torch_dtype=torch.bfloat16,
# )
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", padding_side = "left")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
## Loading the dataset
dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
df = dataset['train_gen'].to_pandas()
display(df)

,prompt,prompt_id,messages
0,How can I fix errors in my grammar when writin...,c033f4b800f390a3e9bb4266d22242c90852a1ef8dee6f...,[{'content': 'How can I fix errors in my gramm...
1,Create a comprehensive guide for writing a pro...,532aec37e5778500677847da77e15ab2d05f40dba9c84d...,[{'content': 'Create a comprehensive guide for...
2,How do people in Hinduism practice daily devot...,173bc31d726a5e8b5e3381a159d98d168a5a94e963449b...,[{'content': 'How do people in Hinduism practi...
3,How does the concept of torsion fields help ex...,74ea19b8ebeeab77180987254f06007fec14ab1d9d8012...,[{'content': 'How does the concept of torsion ...
4,Numerical Modeling of Multi-Species Transport ...,fedf09781f1405da3b2dbc0b5f0a572275b58d2c02e200...,[{'content': 'Numerical Modeling of Multi-Spec...
...,...,...,...
256027,How does the lymphatic system contribute to im...,829f5887f74c1ccdfad6cdf3d96c615fd2eda96c1681b8...,[{'content': 'How does the lymphatic system co...
256028,"Most weekends this semester, you’d find me in ...",fdcd85f6bc6c99ec6f0dd7904103f087e45f2638bea865...,"[{'content': 'Most weekends this semester, you..."
256029,Who were the top contenders in the 1984 U.S. O...,3e98da789057180b68e0bde2e7825dc1f50d6cf80a8956...,[{'content': 'Who were the top contenders in t...
256030,Build a C++ program that calculates the sum of...,8d631e52ca4a32022a7f3d5d2395fe4582211788d78271...,[{'content': 'Build a C++ program that calcula...


In [5]:
## Random Sampling
subset_size = 1500  

# Use the sample() method to select a random subset
df = df.sample(n=subset_size)
df.reset_index(inplace = True)
df = df[['prompt','prompt_id','messages']]
display(df)

,prompt,prompt_id,messages
0,Please create a Python program that can arrang...,f342d434df9d62c2c650c1ae3fc4131c63cf5aba90f3c8...,[{'content': 'Please create a Python program t...
1,Develop a visually appealing and informative p...,32e81af0cec9cdd373d111d9a31ab20b7b34c6e0a1431b...,[{'content': 'Develop a visually appealing and...
2,Provide a list of creative and unique ideas fo...,b06774fb7da80ae9e3e3334111b1a75dbe6dae985ecff8...,[{'content': 'Provide a list of creative and u...
3,Write a comprehensive list of practical and fe...,5e5ba66dc543521db297358a8c2ab3170027d391eba870...,[{'content': 'Write a comprehensive list of pr...
4,Write a short story centered around a mystery ...,85be9fe2b602c32c2b4b8e4b094c57f0d28650b9a62525...,[{'content': 'Write a short story centered aro...
...,...,...,...
1495,Here is a piece of text: A new diabetes vaccin...,c72ad1c88d4f5f0360c7a97b847aece0af3b8ddb5e25d3...,[{'content': 'Here is a piece of text: A new d...
1496,Sorry it’s been so long since we’ve gotten tog...,4c16c4e377ac9355af999922965bbcbc7b12d18f0ca4b1...,[{'content': 'Sorry it’s been so long since we...
1497,Compare and contrast the use of symbolism in S...,81f583efb78216709a5e4209df8ae91073349770b67ddf...,[{'content': 'Compare and contrast the use of ...
1498,Create a 5-paragraph persuasive essay in MLA f...,854d90a3190f7ffdc9153fcd5d010b3915ae4daec47dfe...,[{'content': 'Create a 5-paragraph persuasive ...


In [6]:
## Data Preprocessing

## 1) Separating prompts and responses
# Concatenate all 'content' entries in the arrays
# contents = df['messages'].apply(lambda x: x[1]['content'])
answers = []
for i in range(len(df)):
#     print(i)
    if(len(df.at[i,'messages'])<2):
        df.drop(i,inplace=True)
        
for i in range(len(df)):
    content = df.iat[i,2][1]['content']
    answers.append(content)
# Create a new column in the DataFrame with the concatenated content
df['answer'] = answers

df = df[['prompt','answer']]

## 2) Sorting according to prompt length to incorporate curriculum learning
df['length_col'] = df['prompt'].apply(len)

# Sort the DataFrame by the length column
df_sorted = df.sort_values(by='length_col', ascending=True)  # Use ascending=False for descending order
df = df_sorted[['prompt','answer']]
display(df)

## 3) Removing garbage prompts with very small lengths and hence insufficient context
df=df[476:]
df.reset_index(inplace=True)
df.drop('index',axis=1)
df = df[['prompt','answer']]
display(df)


,prompt,answer
1116,"- 1 red bell pepper, sliced","- 1 yellow bell pepper, sliced\n- 1 orange bel..."
744,- Clear and concise language,- Avoidance of overly technical or jargon-heav...
731,5. Quality of support provided,I am not capable of providing support in the t...
1008,10. Add salt and pepper to taste.,I do not have the ability to taste or add salt...
1216,How can I make a dairy-free curry?,There are several ways to make a dairy-free cu...
...,...,...
43,"Translate the phrase ""composition of paper on ...",La composición de papel sobre varios temas. Nu...
570,Read the passage below and answer the question...,Is there evidence that North Korea has nuclear...
1396,Cerebral somatic oximeter is also known as reg...,"Cerebral somatic oximeters, also known as regi..."
617,I was awarded a PhD in bioinformatics from the...,"The author, who was awarded a PhD in bioinform..."


,prompt,answer
0,Please email a local nail salon and request pe...,Subject: Customized Nail Art Recommendations f...
1,Please share a motivational quote that inspire...,"""The secret of getting ahead is getting starte..."
2,Please provide a list of at least five self-he...,"1. ""The Power of Now"" by Eckhart Tolle - This ..."
3,Write a 10-line rhyming poem in free verse tha...,"Amidst the crowd, I stand alone,\nLost in a se..."
4,Write a step-by-step recipe for a hearty veget...,Hearty Vegetable Soup Recipe\n\nIngredients:\n...
...,...,...
1019,"Translate the phrase ""composition of paper on ...",La composición de papel sobre varios temas. Nu...
1020,Read the passage below and answer the question...,Is there evidence that North Korea has nuclear...
1021,Cerebral somatic oximeter is also known as reg...,"Cerebral somatic oximeters, also known as regi..."
1022,I was awarded a PhD in bioinformatics from the...,"The author, who was awarded a PhD in bioinform..."


In [7]:
## Creating the dataloader class
class Customdataset(Dataset):
    def __init__(self,original_dataset):
        self.original_dataset = original_dataset
    def __len__(self):
        return len(self.original_dataset)
    def __getitem__(self,index):
        prompt = self.original_dataset.iat[index,0]
        response = self.original_dataset.iat[index,1]
        return prompt,response

In [8]:
## Initializing the dataloader
batch_size = 64
d_train = Customdataset(df)
dataloader = DataLoader(d_train, batch_size=batch_size, shuffle=False)

In [9]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [10]:
## Defining the custom tokenizer
def tokenize_and_pad(texts, tokenizer):
    # Tokenize the batch of texts
    #tokenizer.pad_token = tokenizer.eos_token
    tokenized_batches = [tokenizer(batch, return_tensors="pt", padding=False, truncation=True, max_length = 1024) for batch in texts]
    max_length = max(len(text['input_ids'][0]) for text in tokenized_batches)
    tokenized_batches = [tokenizer(batch, return_tensors="pt", padding=False, truncation=True, max_length = max_length) for batch in texts]

    # Pad the sequences with zeros at the end
    for batch in tokenized_batches:
        for key in batch.keys():
    # Calculate the amount of padding needed
            padding_length = max(0, max_length - len(batch[key][0]))
            if key=="attention_mask":
                pad_value = 0
            else:    
                pad_value = tokenizer.convert_tokens_to_ids('<|endoftext|>')  # Assuming you have a tokenizer object

    # Perform left padding with the <s> token
            if padding_length > 0:
                padding_tensor = torch.full((batch[key].shape[0], padding_length), pad_value)

                # Concatenate along the correct dimension
                # If you want to add padding to the right (columns), use dim=1
                batch[key] = torch.cat([ padding_tensor,batch[key]], dim=1)
                
        
    return tokenized_batches

In [11]:
## Using BLEU Score as the evaluation metric
def calculate_bleu_score(paragraph1, paragraph2):
    # Remove tokens in the form of <...> from both paragraphs
    paragraph1_clean = " ".join(word for word in paragraph1.split() if not word.startswith("<") and not word.endswith(">"))
    paragraph2_clean = " ".join(word for word in paragraph2.split() if not word.startswith("<") and not word.endswith(">"))
    
    # Tokenize the paragraphs into lists of words
    reference = nltk.word_tokenize(paragraph1_clean)
    candidate = nltk.word_tokenize(paragraph2_clean)
    
    # Calculate BLEU scores
    bleu_1 = sentence_bleu([reference], candidate, weights=(1, 0, 0, 0),smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
    bleu_2 = sentence_bleu([reference], candidate, weights=(0.5, 0.5, 0, 0),smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
    bleu_3 = sentence_bleu([reference], candidate, weights=(0.33, 0.33, 0.33, 0),smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
    bleu_4 = sentence_bleu([reference], candidate,smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
    
    return bleu_1, bleu_2, bleu_3, bleu_4

In [12]:
## Initializing the optimizer and loading the model
device = "cuda:0"
model.to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-6)
print(model)


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0): Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
      (1): Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(

In [13]:
score_list = []

for step,batch in enumerate(dataloader):
    
    print("Step No "+str(step))
    prompts, ground_truth = batch
    messages = [[{"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}] for prompt in prompts]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True) for message in messages]
    tokenized_batches = tokenize_and_pad(text,tokenizer)
    prompt_ids = torch.stack([x['input_ids'][0].to(device) for x in tokenized_batches], dim = 0)
    prompt_attention_mask = torch.stack([x['attention_mask'][0].to(device) for x in tokenized_batches], dim = 0)
    
    with torch.no_grad():
        model.eval()
        synthetic_response = model.generate(input_ids = prompt_ids, max_new_tokens = 2048)
        output=[output_ids[len(input_ids):] for input_ids, output_ids in zip(tokenized_batches[0].input_ids, synthetic_response)]
        synthetic_response_ids = torch.empty((1,output[0].size(0))).to(device)
        for j in range(batch_size):
            output=[output_ids[len(input_ids):] for input_ids, output_ids in zip(tokenized_batches[j].input_ids, synthetic_response)]
            synthetic_response_ids=torch.cat([synthetic_response_ids.long(),output[0].unsqueeze(0)],dim = 0)
        synthetic_response_ids=synthetic_response_ids[1:,:]
        
        ground_truth_messages = [[{"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": ground_tru}] for ground_tru in ground_truth]
        ground_truth_text = [ tokenizer.apply_chat_template(ground_truth_message,tokenize=False,add_generation_prompt=True) for ground_truth_message in ground_truth_messages]
        
    for i in range(len(synthetic_response_ids)):
        scores = calculate_bleu_score(tokenizer.decode(synthetic_response_ids[i]),ground_truth_text[i])
        score_list.append(scores)
    
    torch.cuda.empty_cache()
    gc.collect()
    model.to(device)
            
avg_bleu_score = sum(score[0] for score in score_list)/len(score_list)
print(f"Average BLEU1 Score is {avg_bleu_score}")
                

Step No 0
Step No 1
Step No 2
Step No 3
Step No 4
Step No 5
Step No 6
Step No 7
Step No 8
Step No 9
Step No 10
Step No 11
Step No 12
Step No 13
Step No 14
Step No 15
Average BLEU1 Score is 0.11013402805278596
